In [1]:
import pandas as pd
import re
from num2words import num2words
import sqlite3

print ('success')

success


In [2]:
conn = sqlite3.connect('binar_gold.db')
cursor = conn.cursor()
print('success')

success


In [3]:
data = pd.read_csv('data.csv',encoding='latin-1')
data.shape

(13169, 13)

In [4]:
cursor.execute('drop table new_kamusalay')
conn.commit()
print('success')

success


In [5]:
alay_df = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
alay_list = alay_df.values.tolist()
print('success')

success


In [6]:
cursor.execute('create table new_kamusalay(alay text, baku text)')
conn.commit()
print('success')

success


In [7]:
for i in alay_list:
    cursor.execute('insert into new_kamusalay(alay,baku) values (?,?)',i)
    conn.commit()

In [8]:
new_kamusalay = pd.read_sql_query('select * from new_kamusalay', conn)
new_kamusalay

,alay,baku
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali
...,...,...
15163,mendag,menteri perdagangan
15164,menaker,menteri tenaga kerja
15165,memetwit,mentwit
15166,megangin,memegang


In [9]:
# show query sql
cursor = conn.execute('select * from new_kamusalay;')
for i in cursor:
    print(i)

('anakjakartaasikasik', 'anak jakarta asyik asyik')
('pakcikdahtua', 'pak cik sudah tua')
('pakcikmudalagi', 'pak cik muda lagi')
('t3tapjokowi', 'tetap jokowi')
('3x', 'tiga kali')
('aamiin', 'amin')
('aamiinn', 'amin')
('aamin', 'amin')
('aammiin', 'amin')
('abis', 'habis')
('abisin', 'habiskan')
('acau', 'kacau')
('achok', 'ahok')
('ad', 'ada')
('adek', 'adik')
('adl', 'adalah')
('adlah', 'adalah')
('adlh', 'adalah')
('ado', 'ada')
('aduhh', 'aduh')
('aer', 'air')
('afdol', 'afdal')
('agamaataualqur', 'agama alquran')
('agm', 'agama')
('agma', 'agama')
('ahaha', 'haha')
('ahahaha', 'haha')
('ahehehehe', 'hehe')
('ahir', 'akhir')
('ahirnya', 'akhirnya')
('ahk', 'ahok')
('ahlamdulillah', 'alhamdulillah')
('ahli2', 'para ahli')
('ahlusunnah', 'ahlus sunah')
('ahmaddani', 'ahmad dhani')
('aho', 'ahok')
('ahoax', 'ahok')
('ahoaxx', 'ahok')
('ahog', 'ahok')
('ahokataudjarot', 'ahok djarot')
('ahokbebanijokowi', 'ahok beban jokowi')
('ahokbtp', 'ahok basuki tjahaja purnama')
('ahokditolakw

In [12]:
# NEW
from flask import Flask, jsonify
from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from

app = Flask(__name__)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\\t|\\n|\\u', ' ', text)
    text = re.sub(r"https?:[^\s]+", ' ', text)
    text = re.sub(r"[^\w\s+]", '', text)
    text = re.sub(r'rt|user', ' ', text)
    text = re.sub(r'[\\x]+[a-z0-9]{2}', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = text.replace('_', ' ')
    text = re.sub(r'(\d+)', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'(\d+)', lambda x: num2words(int(x.group(0)), lang='id'), text)
    return text

alay_filter = dict(zip(new_kamusalay['alay'], new_kamusalay['baku']))

def normalisasi_alay(text):
    return ' '.join(alay_filter.get(word, word) for word in text.split(' '))

app.json_encoder = LazyJSONEncoder
swagger_template = dict(
info = {
    'title': 'API Documentation for Data Processing and Modeling',
    'version': '1.0.0',
    'description': 'Dokumentasi API untuk Data Processing dan Modeling',
    },
#     host = "127.0.0.1:5000"
#     host = LazyString(lambda: request.host)
)

swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}

swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)

@swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/hello_world.yml", methods=['GET'])
@app.route('/', methods=['GET'])
def hello_world():
    json_response = {
        'status_code': 200,
        'description': "Menyapa Hello World",
        'data': "Hello World",
    }

    response_data = jsonify(json_response)
    return response_data

# @swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/text.yml", methods=['GET'])
# @app.route('/text', methods=['GET'])
# def text():
#     json_response = {
#         'status_code': 200,
#         'description': "Original Teks",
#         'data': "Halo, apa kabar semua?",
#     }

#     response_data = jsonify(json_response)
#     return response_data

# @swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/text_clean.yml", methods=['GET'])
# @app.route('/text-clean', methods=['GET'])
# def text_clean():
#     json_response = {
#         'status_code': 200,
#         'description': "Teks yang sudah dibersihkan",
#         'data': re.sub(r'[^a-zA-Z0-9]', ' ', "Halo, apa kabar semua?"),
#     }

#     response_data = jsonify(json_response)
#     return response_data

@swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/text_processing.yml", methods=['POST'])
@app.route('/text-processing', methods=['POST'])
def text_processing():

    text = request.form.get('text')
    cleaned_text = clean_text(text)
    normalized_text = normalisasi_alay(cleaned_text)

    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': normalized_text,
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/text_processing_file.yml", methods=['POST'])
@app.route('/text-processing-file', methods=['POST'])
def text_processing_file():

    # Upladed file
    file = request.files.getlist('file')[0]

    # Import file csv ke Pandas
    data = pd.read_csv('data.csv',encoding='latin-1')

    # Ambil teks yang akan diproses dalam format list
    texts = data.Tweet.to_list()

    # Lakukan cleansing pada teks
    cleaned_text = []
    for text in texts:
        text = clean_text(text)
        text = normalisasi_alay(text)
        cleaned_text.append(text)
    
    # koneksi dengan database
    conn = sqlite3.connect('binar_gold.db')
    cursor = conn.cursor()

    # membuat table cleaned_text 
    cursor.execute('create table if not exists cleaned_text(id integer primary key autoincrement, text text)')
    
    # insert data di table cleaned_text
    for text in cleaned_text:
        cursor.execute('insert into cleaned_text(text) values (?)', (text,))
    
    conn.commit()
    
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': cleaned_text,
    }

    response_data = jsonify(json_response)
    return response_data

if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Mar/2024 12:34:30] "GET /docs/ HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 12:34:30] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Mar/2024 12:34:30] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Mar/2024 12:34:30] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Mar/2024 12:34:30] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Mar/2024 12:34:30] "GET /docs.json HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 12:34:30] "GET /flasgger_static/favicon-32x32.png HTTP/1.1" 304 -
127.0.0.1 - - [21/Mar/2024 12:34:40] "POST /text-processing-file HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 12:37:08] "POST /text-processing HTTP/1.1" 200 -


In [13]:
# cursor = conn.execute('select * from new_kamusalay;')
# for i in cursor:
#     print(i)
    
cursor = conn.execute('select * from cleaned_text')
for i in cursor:
    print(i)

(1, 'di saat semua cowok berusaha melacak perhatian aku kamu lantas remehkan perhatian yang aku kasih khusus ke kamu basic kamu cowok bego')
(2, 'siapa yang telat memberi tau eluedan sarap aku bergaul dengan cigax jifla calis sama siapa itu licew juga')
(3, 'empat puluh satu kadang aku berpikir kenapa aku tetap percaya pada tuhan padahal aku selalu jatuh berkalikali kadang aku merasa tuhan itu meninggalkan aku sendirian ketika orang tuaku berencana berpisah ketika kakakku lebih memilih jadi kristen ketika aku anak ter')
(4, 'aku itu aku ku tau matamu sipit tapi dilihat dari mana itu aku')
(5, 'kaum cebong kafir sudah kelihatan dongoknya dari awal tambah dungu lagi haha')
(6, 'ya bani taplak dan kawan kawan')
(7, 'deklarasi pilihan kepala daerah dua ribu delapan belas aman dan anti hoaks warga dukuh sari jabon')
(8, 'aku baru saja selesai rewatch aldnoah zero paling kampret memang akhirnya dua karakter utama cowoknya kena friendzone bro xd uniform resource locator')
(9, 'nah admin belan

In [14]:
cleaned_text = pd.read_sql_query('select * from cleaned_text',conn)
cleaned_text.head()

,id,text
0,1,di saat semua cowok berusaha melacak perhatian...
1,2,siapa yang telat memberi tau eluedan sarap aku...
2,3,empat puluh satu kadang aku berpikir kenapa ak...
3,4,aku itu aku ku tau matamu sipit tapi dilihat d...
4,5,kaum cebong kafir sudah kelihatan dongoknya da...
